In [1]:
import numpy as np
import matplotlib.pyplot as plt
from spectral import *
from scipy import linalg
import scipy.stats as stats
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import euclidean
from scipy.stats import f_oneway
import pandas as pd
import cv2
import glob
import os

In [3]:
data_dir = glob.glob(r"C:\Users\htic\Documents\MSI_dataset - Copy\data\*\*.hdr")
mask_files = glob.glob(r"C:\Users\htic\Documents\MSI_dataset\masks\*.png")

msi = []
name_msi = []

masks_list = []
name_mask = []
for data in data_dir:
    msi.append(open_image(data))
    name_msi.append(os.path.basename(data))
print(len(msi))


for mask_file in mask_files:
    mask_data = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    _, mask_data = cv2.threshold(mask_data, 0, 1, cv2.THRESH_BINARY)
    masks_list.append(mask_data)
    mask_filename = os.path.basename(mask_file)
    name_mask.append(mask_filename)

for i in range(len(name_msi)):
    print(f"Data - {name_msi[i]} \t Mask - {name_mask[i]}")

185
Data - 1.hdr 	 Mask - 1.png
Data - 10.hdr 	 Mask - 10.png
Data - 100.hdr 	 Mask - 100.png
Data - 103.hdr 	 Mask - 103.png
Data - 104.hdr 	 Mask - 104.png
Data - 105.hdr 	 Mask - 105.png
Data - 106.hdr 	 Mask - 106.png
Data - 109.hdr 	 Mask - 109.png
Data - 110.hdr 	 Mask - 110.png
Data - 111.hdr 	 Mask - 111.png
Data - 112.hdr 	 Mask - 112.png
Data - 113.hdr 	 Mask - 113.png
Data - 115.hdr 	 Mask - 115.png
Data - 116.hdr 	 Mask - 116.png
Data - 117.hdr 	 Mask - 117.png
Data - 118.hdr 	 Mask - 118.png
Data - 119.hdr 	 Mask - 119.png
Data - 12.hdr 	 Mask - 12.png
Data - 120.hdr 	 Mask - 120.png
Data - 121.hdr 	 Mask - 121.png
Data - 123.hdr 	 Mask - 123.png
Data - 124.hdr 	 Mask - 124.png
Data - 125.hdr 	 Mask - 125.png
Data - 126.hdr 	 Mask - 126.png
Data - 127.hdr 	 Mask - 127.png
Data - 128.hdr 	 Mask - 128.png
Data - 129.hdr 	 Mask - 129.png
Data - 13.hdr 	 Mask - 13.png
Data - 133.hdr 	 Mask - 133.png
Data - 14.hdr 	 Mask - 14.png
Data - 142.hdr 	 Mask - 142.png
Data - 143.hdr 	

In [9]:
def analyser(img, masks):
    band_data = []
    spectral_image_mean = []
    for k in range(16):
        band = img[:,:,k]
        #Mask Multiplication
        masking = cv2.bitwise_and(band, band, mask=mask)
        unmasked_area = masking[np.nonzero(masking)]
        band_data.append(np.mean(unmasked_area))

    #normalizing the band data in range 0-1
    norm_band_data = (band_data - np.min(band_data))/(np.max(band_data - np.min(band_data)))
    spectral_image_mean.append(np.array(norm_band_data))

    return spectral_image_mean

In [16]:
spectral_data = []
for img, mask in zip(msi, masks_list):
    spectral_data.append(analyser(img, mask))

In [26]:
spectral_data = np.reshape(spectral_data, (185, 16))
spec_df = pd.DataFrame(spectral_data, columns=[str(f"Band {i+1}") for i in range(16)])
spec_df.insert(0, "ID", value = name_msi)
# Save the DataFrame as an Excel file
spec_df.to_excel("spectrum.xlsx", index=False)
# Print the DataFrame
print(spec_df)


          ID    Band 1    Band 2    Band 3    Band 4    Band 5    Band 6  \
0      1.hdr  0.191421  0.174875  0.055949  0.000000  0.112394  0.239586   
1     10.hdr  0.796949  0.425522  0.098905  0.000000  0.010546  0.131317   
2    100.hdr  0.430056  0.140397  0.320150  0.636122  0.475299  0.300668   
3    103.hdr  0.114321  0.156568  0.006411  0.000000  0.141338  0.160023   
4    104.hdr  1.000000  0.875767  0.740997  0.753059  0.469438  0.364658   
..       ...       ...       ...       ...       ...       ...       ...   
180   95.hdr  0.118435  0.124986  0.055592  0.104634  0.152587  0.161478   
181   96.hdr  0.102753  0.092704  0.012496  0.000000  0.468702  0.499503   
182   97.hdr  0.559110  0.465326  0.443695  0.522629  0.358354  0.363387   
183   98.hdr  0.673073  0.419508  0.332230  0.088049  0.146953  0.351314   
184   99.hdr  1.000000  0.618728  0.240356  0.235733  0.360017  0.540275   

       Band 7    Band 8    Band 9   Band 10   Band 11   Band 12   Band 13  \
0    0.306

In [18]:
spec_df = pd.DataFrame(np.array(spectral_data), columns = ('{}'.format(i) for i in range(0,16)))
spec_df
# spec_df["ID"] = name_msi

ValueError: Must pass 2-d input. shape=(185, 1, 16)

In [ ]:
print(name_msi)
print(name_mask)
print(f"lenghts of data = {len(name_msi)}, masks = {len(name_mask)}")

NameError: name 'name_msi' is not defined

185


In [ ]:
class DataProcessor:
    def __init__(self, leuko_data_dir, leuko_masks_dir, leuko_rgbs_dir, scc_data_dir, scc_masks_dir, scc_rgbs_dir, osmf_data_dir, osmf_masks_dir, osmf_rgbs_dir,
                 dh_leuko_data_dir, dh_leuko_masks_dir, dh_leuko_rgbs_dir, dh_scc_data_dir, dh_scc_masks_dir, dh_scc_rgbs_dir, dh_osmf_data_dir, dh_osmf_masks_dir, dh_osmf_rgbs_dir,
                 hh_data_dir, hh_masks_dir, hh_rgbs_dir):
                  
        # Leuko Data
        self.leuko_data_dir = leuko_data_dir
        self.leuko_masks_dir = leuko_masks_dir
        self.leuko_rgbs_dir = leuko_rgbs_dir

        # SCC Data
        self.scc_data_dir = scc_data_dir
        self.scc_masks_dir = scc_masks_dir
        self.scc_rgbs_dir = scc_rgbs_dir

        # OSMF Data
        self.osmf_data_dir = osmf_data_dir
        self.osmf_masks_dir = osmf_masks_dir
        self.osmf_rgbs_dir = osmf_rgbs_dir

        # Disease Healthy - Leuko
        self.dh_leuko_data_dir= dh_leuko_data_dir
        self.dh_leuko_masks_dir = dh_leuko_masks_dir
        self.dh_leuko_rgbs_dir= dh_leuko_rgbs_dir

        #Disease Healthy - SCC
        self.dh_scc_data_dir= dh_scc_data_dir
        self.dh_scc_masks_dir = dh_scc_masks_dir
        self.dh_scc_rgbs_dir= dh_scc_rgbs_dir
        
        #Disease Healthy - OSMF
        self.dh_osmf_data_dir= dh_osmf_data_dir
        self.dh_osmf_masks_dir = dh_osmf_masks_dir
        self.dh_osmf_rgbs_dir= dh_osmf_rgbs_dir

        # Healthy Data
        self.hh_data_dir = hh_data_dir
        self.hh_masks_dir = hh_masks_dir
        self.hh_rgbs_dir = hh_rgbs_dir

        self.leuko_imgs = []
        self.leuko_masks = []
        self.leuko_rgbs = []
        self.leuko_data_name = []
        self.leuko_masks_name = []
        self.leuko_rgbs_name = []

        self.scc_imgs = []
        self.scc_masks = []
        self.scc_rgbs = []
        self.scc_data_name = []
        self.scc_masks_name = []
        self.scc_rgbs_name = []

        self.osmf_imgs = []
        self.osmf_masks = []
        self.osmf_rgbs = []
        self.osmf_data_name = []
        self.osmf_masks_name = []
        self.osmf_rgbs_name = []

        self.dh_leuko_imgs = []
        self.dh_leuko_masks = []
        self.dh_leuko_rgbs = []    
        self.dh_leuko_data_name = []
        self.dh_leuko_masks_name = []
        self.dh_leuko_rgbs_name = []

        self.dh_scc_imgs = []
        self.dh_scc_masks = []
        self.dh_scc_rgbs = [] 
        self.dh_scc_data_name = []
        self.dh_scc_masks_name = []
        self.dh_scc_rgbs_name = []

        self.dh_osmf_imgs = []
        self.dh_osmf_masks = []
        self.dh_osmf_rgbs = []        
        self.dh_osmf_data_name = []
        self.dh_osmf_masks_name = []
        self.dh_osmf_rgbs_name = []

        self.hh_imgs = []
        self.hh_masks = []
        self.hh_rgbs = []
        self.hh_data_name = []
        self.hh_masks_name = []
        self.hh_rgbs_name = []

    def load_multispectral_data(self, data_dir, imgs_list, data_name):
        # msi_files = data_dir 
        msi_files = glob.glob(os.path.join(data_dir))
        for msi_file in msi_files:
            img_data = open_image(msi_file)
            imgs_list.append(np.asarray(img_data.load()))
            img_filename = os.path.basename(msi_file)
            data_name.append(img_filename)

    def load_masks(self, masks_dir, masks_list, masks_name):
        # mask_files = masks_dir
        mask_files = glob.glob(os.path.join(masks_dir))
        for mask_file in mask_files:
            mask_data = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
            _, mask_data = cv2.threshold(mask_data, 0, 1, cv2.THRESH_BINARY)
            masks_list.append(mask_data)
            mask_filename = os.path.basename(mask_file)
            masks_name.append(mask_filename)

    def load_rgbs(self, rgbs_dir, rgbs_list, rgbs_name):
        # rgb_files = rgbs_dir
        rgb_files = glob.glob(os.path.join(rgbs_dir))
        for rgb_file in rgb_files:
            rgb_im = cv2.imread(rgb_file)
            rgb_im = cv2.cvtColor(rgb_im, cv2.COLOR_BGR2RGB)
            rgbs_list.append(rgb_im)
            rgb_filename = os.path.basename(rgb_file)
            rgbs_name.append(rgb_filename)

    def process_data(self):
        self.load_multispectral_data(self.leuko_data_dir, self.leuko_imgs, self.leuko_data_name)
        self.load_masks(self.leuko_masks_dir, self.leuko_masks, self.leuko_masks_name)
        self.load_rgbs(self.leuko_rgbs_dir, self.leuko_rgbs, self.leuko_rgbs_name)

        self.load_multispectral_data(self.scc_data_dir, self.scc_imgs, self.scc_data_name)
        self.load_masks(self.scc_masks_dir, self.scc_masks, self.scc_masks_name)
        self.load_rgbs(self.scc_rgbs_dir, self.scc_rgbs, self.scc_rgbs_name)

        self.load_multispectral_data(self.osmf_data_dir, self.osmf_imgs, self.osmf_data_name)
        self.load_masks(self.osmf_masks_dir, self.osmf_masks, self.osmf_masks_name)
        self.load_rgbs(self.osmf_rgbs_dir, self.osmf_rgbs, self.osmf_rgbs_name)

        self.load_multispectral_data(self.dh_leuko_data_dir, self.dh_leuko_imgs, self.dh_leuko_data_name)
        self.load_masks(self.dh_leuko_masks_dir, self.dh_leuko_masks, self.dh_leuko_masks_name)
        self.load_rgbs(self.dh_leuko_rgbs_dir, self.dh_leuko_rgbs, self.dh_leuko_rgbs_name)

        self.load_multispectral_data(self.dh_scc_data_dir, self.dh_scc_imgs, self.dh_scc_data_name)
        self.load_masks(self.dh_scc_masks_dir, self.dh_scc_masks, self.dh_scc_masks_name)
        self.load_rgbs(self.dh_scc_rgbs_dir, self.dh_scc_rgbs, self.dh_scc_rgbs_name)

        self.load_multispectral_data(self.dh_osmf_data_dir, self.dh_osmf_imgs, self.dh_osmf_data_name)
        self.load_masks(self.dh_osmf_masks_dir, self.dh_osmf_masks, self.dh_osmf_masks_name)
        self.load_rgbs(self.dh_osmf_rgbs_dir, self.dh_osmf_rgbs, self.dh_osmf_rgbs_name)

        self.load_multispectral_data(self.hh_data_dir, self.hh_imgs, self.hh_data_name)
        self.load_masks(self.hh_masks_dir, self.hh_masks, self.hh_masks_name)
        self.load_rgbs(self.hh_rgbs_dir, self.hh_rgbs, self.hh_rgbs_name)


In [ ]:
# Create an instance of the DataProcessor class and process the data
base_dir = "C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/"
data_processor = DataProcessor(
    
    # Read Leuko data
    leuko_data_dir = os.path.join(base_dir, "UH/class/leuko/data/*.hdr"), 
    leuko_masks_dir = os.path.join(base_dir, "UH/class/leuko/masked/*.png"),
    leuko_rgbs_dir = os.path.join(base_dir, "UH/class/leuko/rgb/*.png"),

    # Read OSCC data
    scc_data_dir = os.path.join(base_dir, "UH/class/oscc/data/*.hdr"),
    scc_masks_dir = os.path.join(base_dir, "UH/class/oscc/masked/*.png"),
    scc_rgbs_dir = os.path.join(base_dir, "UH/class/oscc/rgb/*.png"),

    # Read OSMF data
    osmf_data_dir = os.path.join(base_dir, "UH/class/osmf/data/*.hdr"),
    osmf_masks_dir = os.path.join(base_dir, "UH/class/osmf/masked/*.png"),
    osmf_rgbs_dir = os.path.join(base_dir, "UH/class/osmf/rgb/*.png"),

    # Read DH-Leuko data
    dh_leuko_data_dir = os.path.join(base_dir, "DH/DH/leuko/data/*.hdr"),
    dh_leuko_masks_dir = os.path.join(base_dir, "DH/DH/leuko/masked/*.png"),
    dh_leuko_rgbs_dir = os.path.join(base_dir, "DH/DH/leuko/rgb/*.png"),

    # Read DH-OSCC data
    dh_scc_data_dir = os.path.join(base_dir, "DH/DH/oscc/data/*.hdr"),
    dh_scc_masks_dir = os.path.join(base_dir, "DH/DH/oscc/masked/*.png"),
    dh_scc_rgbs_dir = os.path.join(base_dir, "DH/DH/oscc/rgb/*.png"),

    # Read DH-OSMF data
    dh_osmf_data_dir = os.path.join(base_dir, "DH/DH/osmf/data/*.hdr"),
    dh_osmf_masks_dir = os.path.join(base_dir, "DH/DH/osmf/masked/*.png"),
    dh_osmf_rgbs_dir = os.path.join(base_dir, "DH/DH/osmf/rgb/*.png"),

    # Read Healthy data
    hh_data_dir = os.path.join(base_dir, "HH/HH/data/*.hdr"),
    hh_masks_dir = os.path.join(base_dir, "HH/HH/masked/*.png"),
    hh_rgbs_dir = os.path.join(base_dir, "HH/HH/rgbs/*.png"),
)
data_processor.process_data()


In [ ]:
# To verify with the excel is all the respective data are read - Leuko
print(f"Leuko Data - {len(data_processor.leuko_data_name)}")
for i in range(len(data_processor.leuko_data_name)):
    # print(len(data_processor.leuko_data_name))
    print(f"{i}. Data = {data_processor.leuko_data_name[i]}, Mask = {data_processor.leuko_masks_name[i]}, RGB = {data_processor.leuko_rgbs_name[i]}")

In [ ]:
# To verify with the excel is all the respective data are read - OSCC
print(f"SCC Data - {len(data_processor.scc_data_name)}")
for i in range(len(data_processor.scc_data_name)):
    print(f"{i}. Data = {data_processor.scc_data_name[i]}, Mask = {data_processor.scc_masks_name[i]}, RGB = {data_processor.scc_rgbs_name[i]}")

In [ ]:
# To verify with the excel is all the respective data are read - DH-Leuko
print(f"Disease-healthy Data - {len(data_processor.dh_leuko_data_name)}")
for i in range(len(data_processor.dh_leuko_data_name)):
    print(f"{i}. Data = {data_processor.dh_leuko_data_name[i]}, Mask = {data_processor.dh_leuko_data_name[i]}, RGB = {data_processor.dh_leuko_data_name[i]}")

In [ ]:
# To verify with the excel is all the respective data are read - DH-OSCC
print(f"Disease-healthy Data - {len(data_processor.dh_scc_data_name)}")
for i in range(len(data_processor.dh_scc_data_name)):
    print(f"{i}. Data = {data_processor.dh_scc_data_name[i]}, Mask = {data_processor.dh_scc_data_name[i]}, RGB = {data_processor.dh_scc_data_name[i]}")

In [ ]:
# To verify with the excel is all the respective data are read 
print(f"\nHealthy Data - {len(data_processor.hh_data_name)}")
for i in range(len(data_processor.hh_data_name)):
    print(f"{i}. Data = {data_processor.hh_data_name[i]}, Mask = {data_processor.hh_masks_name[i]}, RGB = {data_processor.hh_rgbs_name[i]}")

In [ ]:
# #548 and 576
# #Band 10 and 14
# def oxy_cal(img):
# #    # R542 = 52096 , 49708
# #    # R572 = 49172, 43340  
#     attmat = np.array([[52096,49708],[49172,43340]])

#     lnR544 = np.log(img[:,:,10])  #changing reflectance to absorbance
#     lnR572 = np.log(img[:,:,14])  #800

#     has_inf = np.isinf(lnR544).any()

#     if has_inf:
#         print("The image contains inf values.")
 
#     lnR544 = np.nan_to_num(lnR544)
#     lnR572 = np.nan_to_num(lnR572)

#     hbO2 = []
#     Hbb = []
#     for a1,a2 in zip(lnR544,lnR572): #a1,a2 has the value of each rows from ln545, ln800 respectively
#         for i,j in zip(a1,a2):
#             rest= np.array([[i],[j]]) #i and j will have 1 pixel value from ln544, ln572 respectively
#             y = linalg.solve(attmat, rest)
#             hbo2 = y[0]
#             hb = y[1]
#             hbO2.append(hbo2)
#             Hbb.append(hb)

#     HbO2 = np.array(hbO2).reshape(270,510)
#     Hb= np.array(Hbb).reshape(270,510)

#     #normalizing Hb, Hb02 values before calculating st02
#     n_Hb = ((Hb- Hb.min())/(Hb.max()-Hb.min()))*100
#     n_HbO2 = ((HbO2- HbO2.min())/(HbO2.max()-HbO2.min()))*100

#     StO2 = n_HbO2/(n_Hb+n_HbO2)
#     n_StO2 = ((StO2 - StO2.min()))/(StO2.max() - StO2.min())*100        
#     return n_Hb, n_HbO2, n_StO2

In [ ]:
def oxy_cal(img):
   # R542 = 52096 , 49708
   # R572 = 49172, 43340 
    attmat = np.array([[52096,49708],[49172,43340]])

    lnR544 = np.log(img[:,:,10])  #changing reflectance to absorbance
    lnR572 = np.log(img[:,:,14])  #800

    has_inf = np.isinf(lnR544).any()

    if has_inf:
        print("The image contains inf values.")
 
    lnR544 = np.nan_to_num(lnR544)
    lnR572 = np.nan_to_num(lnR572)

    # hbO2 = []
    # Hbb = []
    lnR544 = lnR544.reshape(270*510)
    lnR572 = lnR572.reshape(270*510)
    
    y = np.linalg.lstsq(attmat, [lnR544, lnR572])[0]
    hbo2 = y[0]
    hb = y[1]
    hbO2 = hbo2.reshape(270,510)
    Hbb = hb.reshape(270,510)

    #normalizing Hb, Hb02 values before calculating st02
    n_Hb = ((Hbb- Hbb.min())/(Hbb.max()-Hbb.min())) * 100
    n_HbO2 = ((hbO2- hbO2.min())/(hbO2.max()-hbO2.min())) * 100

    StO2 = n_HbO2/(n_Hb+n_HbO2)
    n_StO2 = ((StO2 - StO2.min()))/(StO2.max() - StO2.min()) * 100        
    return n_Hb, n_HbO2, n_StO2

In [ ]:
def analyser(rgbs, imgs, masks, patho):
    #for naming of the plot
    if patho == 0:
        rgbs_name_list = data_processor.leuko_rgbs_name
        masks_name_list = data_processor.leuko_masks_name
        data_name_list = data_processor.leuko_data_name
    elif patho == 1:
        rgbs_name_list = data_processor.scc_rgbs_name
        masks_name_list = data_processor.scc_masks_name
        data_name_list = data_processor.scc_data_name
    elif patho == 2:
        rgbs_name_list = data_processor.osmf_rgbs_name
        masks_name_list = data_processor.osmf_masks_name
        data_name_list = data_processor.osmf_data_name
    elif patho == 3:
        rgbs_name_list = data_processor.dh_leuko_rgbs_name
        masks_name_list = data_processor.dh_leuko_masks_name
        data_name_list = data_processor.dh_leuko_data_name
    elif patho == 4:
        rgbs_name_list = data_processor.dh_scc_rgbs_name
        masks_name_list = data_processor.dh_scc_masks_name
        data_name_list = data_processor.dh_scc_data_name
    elif patho == 5:
        rgbs_name_list = data_processor.dh_osmf_rgbs_name
        masks_name_list = data_processor.dh_osmf_masks_name
        data_name_list = data_processor.dh_osmf_data_name
    elif patho == 6:
        rgbs_name_list = data_processor.hh_rgbs_name
        masks_name_list = data_processor.hh_masks_name
        data_name_list = data_processor.hh_data_name


    mean_StO2 = []
    spectral_image_mean = []

    for i in range(len(imgs)):
        image = imgs[i]
        mask = masks[i]
        rgb = rgbs[i]

        Hb, HbO2, StO2 = oxy_cal(image)
        masked_data = cv2.bitwise_and(StO2, StO2, mask = mask)
        # plt.figure(figsize=(45, 15))
        # plt.subplot(1,3,1), plt.imshow(rgb), plt.title(f"RGB of {rgbs_name_list[i]}"),  plt.axis("off")
        # plt.subplot(1,3,3), plt.imshow(masked_data, cmap= 'jet'), plt.title(f"Masked Area StO2 {masks_name_list[i]}"),  plt.axis("off")
        # plt.subplot(1,3,2), plt.imshow(StO2, cmap="jet"), plt.title(f"StO2 of {data_name_list[i]}"), plt.axis("off")
        # name = data_name_list[i]
        # # plt.savefig(f"C:/Users/htic/Desktop/StO2_images/Healthy/{name}.png")
        # plt.show()
        unmasked_pixels = masked_data[np.nonzero(masked_data)]
        mean_StO2.append(np.mean(unmasked_pixels))

        band_data = []
        for k in range(16):
            band = image[:,:,k]
            #Mask Multiplication
            masking = cv2.bitwise_and(band, band, mask=mask)
            unmasked_area = masking[np.nonzero(masking)]
            band_data.append(np.mean(unmasked_area))

        #normalizing the band data in range 0-1
        norm_band_data = (band_data - np.min(band_data))/(np.max(band_data - np.min(band_data)))
        spectral_image_mean.append(np.array(norm_band_data))

    return mean_StO2, spectral_image_mean


In [ ]:
#Calculating the StO2 and generating spectral graph - Leuko
leuko_mean_StO2, leuko_spectral_img_mean  = analyser(data_processor.leuko_rgbs, data_processor.leuko_imgs, data_processor.leuko_masks, 0)

In [ ]:
#Calculating the StO2 and generating spectral graph - SCC
scc_mean_StO2, scc_spectral_img_mean  = analyser(data_processor.scc_rgbs, data_processor.scc_imgs, data_processor.scc_masks, 1)

In [ ]:
#Calculating the StO2 and generating spectral graph - DH-Leuko
dh_leuko_mean_StO2, dh_leuko_spectral_img_mean  = analyser(data_processor.dh_leuko_rgbs, data_processor.dh_leuko_imgs, data_processor.dh_leuko_masks, 3)

In [ ]:
#Calculating the StO2 and generating spectral graph - DH-SCC
dh_scc_mean_StO2, dh_scc_spectral_img_mean  = analyser(data_processor.dh_scc_rgbs, data_processor.dh_scc_imgs, data_processor.dh_scc_masks, 4)

In [ ]:
#Calculating the StO2 and generating spectral graph - Healthy
hh_mean_StO2, hh_spectral_img_mean = analyser(data_processor.hh_rgbs, data_processor.hh_imgs, data_processor.hh_masks, 6)

In [ ]:
"""
To save the all the mean spectra in one csv file with disease code.
No need to run again as it is saved already. 

"""
leuko_mean = pd.DataFrame(np.array(leuko_spectral_img_mean),  columns = ('{}'.format(i) for i in range(0,16)))
# leuko_mean.insert(loc=2, column="Task", value = data_processor.leuko_data_name)
leuko_mean.insert(loc=0, column='Code', value=0)
leuko_mean.insert(loc=1, column='Patho', value= "L")
leuko_mean['Task'] = data_processor.leuko_data_name

scc_mean = pd.DataFrame(np.array(scc_spectral_img_mean),  columns = ('{}'.format(i) for i in range(0,16)))
scc_mean.insert(loc=0, column='Code', value=1)
scc_mean.insert(loc=1, column='Patho', value= "OSCC")
scc_mean['Task'] = data_processor.scc_data_name


dh_leuko_mean = pd.DataFrame(np.array(dh_leuko_spectral_img_mean), columns = ('{}'.format(i) for i in range(0,16)))
dh_leuko_mean.insert(loc=0, column='Code', value=2)
dh_leuko_mean['Task'] = data_processor.dh_leuko_data_name
dh_leuko_mean.insert(loc=1, column='Patho', value= "AL")

dh_scc_mean = pd.DataFrame(np.array(dh_scc_spectral_img_mean), columns = ('{}'.format(i) for i in range(0,16)))
dh_scc_mean.insert(loc=0, column='Code', value=3)
dh_scc_mean['Task'] = data_processor.dh_scc_data_name
dh_scc_mean.insert(loc=1, column='Patho', value= "ASC")

hh_mean = pd.DataFrame(np.array(hh_spectral_img_mean),  columns = ('{}'.format(i) for i in range(0,16)))
hh_mean.insert(loc=0, column='Code', value=4)
hh_mean['Task'] = data_processor.hh_data_name
hh_mean.insert(loc=1, column='Patho', value= "H")

dataframes = [leuko_mean, scc_mean, dh_leuko_mean, dh_scc_mean, hh_mean]
combined_df = pd.concat(dataframes)

# csv_path = "C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/five.csv"
# combined_df.to_csv(csv_path, index=False)



In [ ]:
# Plotting of Leuko spectral data
# for i in range(len(leuko_spectral_img_mean)):
#     plt.plot(leuko_spectral_img_mean[i])
#     name = data_processor.leuko_data_name[i]
#     plt.title(f"Spectral Graph -Leuko name{name}")
#     # plt.savefig(f"C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/{name}.png")
#     plt.show()

# Create a DataFrame from the leuko spectral data

leuko_data = []
for i in range(len(leuko_spectral_img_mean)):
    leuko_spectra = np.array(leuko_spectral_img_mean[i])
    leuko_spectra = leuko_spectra.T
    leuko_data.append(leuko_spectra)

leuko_data = np.vstack(leuko_data)
leuko_df = pd.DataFrame(leuko_data, columns = ('{}'.format(i) for i in range(0,16)))
leuko_df.insert(loc=0, column='Code', value=0)
# leuko_df.insert(loc=1, column='Patho', value= "Disease")
leuko_df.insert(loc=1, column='Patho', value= "L")


In [ ]:
#Plotting of SCC spectral data
# for i in range(len(scc_spectral_img_mean)):
#     plt.plot(scc_spectral_img_mean[i])
#     name = data_processor.scc_data_name[i]
#     plt.title(f"Spectral Graph -Leuko name{name}")
#     # plt.savefig(f"C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Spectras/SCC/{name}.png")
#     plt.show()

# Create a DataFrame from the leuko spectral data
scc_data = []
for i in range(len(scc_spectral_img_mean)):
    scc_spectra = np.array(scc_spectral_img_mean[i])
    scc_spectra = scc_spectra.T
    scc_data.append(scc_spectra)

scc_data = np.vstack(scc_data)
scc_df = pd.DataFrame(scc_data, columns = ('{}'.format(i) for i in range(0,16)))
scc_df.insert(loc=0, column='Code', value=0)
# scc_df.insert(loc=1, column='Patho', value= "Disease")
scc_df.insert(loc=1, column='Patho', value= "OSCC")

In [ ]:
#Plotting of DH-Leuko spectral data
# for i in range(len(dh_leuko_spectral_img_mean)):
#     plt.plot(dh_leuko_spectral_img_mean[i])
#     name = data_processor.dh_leuko_data_name[i]
#     plt.title(f"Spectral Graph -Leuko name{name}")
#     # plt.savefig(f"C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Spectras/DH-Leuko/{name}.png")
#     plt.show()

# Create a DataFrame from the leuko spectral data
dh_leuko_data = []
for i in range(len(dh_leuko_spectral_img_mean)):
    dh_leuko_spectra = np.array(dh_leuko_spectral_img_mean[i])
    dh_leuko_spectra = dh_leuko_spectra.T
    dh_leuko_data.append(dh_leuko_spectra)

dh_leuko_data = np.vstack(dh_leuko_data)
dh_leuko_df = pd.DataFrame(dh_leuko_data, columns = ('{}'.format(i) for i in range(0,16)))
# dh_leuko_df.insert(loc=0, column='Code', value=0)
dh_leuko_df.insert(loc=1, column='Patho', value= "AL")

In [ ]:
#Plotting of DH-SCC spectral data
# for i in range(len(dh_scc_spectral_img_mean)):
#     plt.plot(dh_scc_spectral_img_mean[i])
#     name = data_processor.dh_scc_data_name[i]
#     plt.title(f"Spectral Graph -Leuko name{name}")
#     # plt.savefig(f"C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Spectras/DH-SCC/{name}.png")
#     plt.show()

# Create a DataFrame from the leuko spectral data
dh_scc_data = []
for i in range(len(dh_scc_spectral_img_mean)):
    dh_scc_spectra = np.array(dh_scc_spectral_img_mean[i])
    dh_scc_spectra = dh_scc_spectra.T
    dh_scc_data.append(dh_scc_spectra)

dh_scc_data = np.vstack(dh_scc_data)
print(dh_scc_data.shape)
dh_scc_df = pd.DataFrame(dh_scc_data, columns = ('{}'.format(i) for i in range(0,16)))
dh_scc_df.insert(loc=0, column='Code', value=0)
# dh_scc_df.insert(loc=1, column='Patho', value= "Disease")
dh_scc_df.insert(loc=1, column='Patho', value= "ASC")


In [ ]:
# Plotting of Healthy spectral data
# for i in range(len(hh_spectral_img_mean)):
#     plt.plot(hh_spectral_img_mean[i])
#     name = data_processor.hh_data_name[i]
#     plt.title(f"Spectral Graph -Leuko name{name}")
#     # plt.savefig(f"C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Spectras/HH/{name}.png")
#     plt.show()
hh_data = []
for i in range(len(hh_spectral_img_mean)):
    hh_spectra = np.array(hh_spectral_img_mean[i])
    hh_spectra = hh_spectra.T
    hh_data.append(hh_spectra)

hh_data = np.vstack(hh_data)
print(hh_data.shape)
hh_df = pd.DataFrame(hh_data, columns = ('{}'.format(i) for i in range(0,16)))
hh_df.insert(loc=0, column='Code', value=1)
hh_df.insert(loc=1, column='Patho', value= "Healthy")

########## Saving Data Frames in CSV and Excel file ##########
####################### #Already Saved #########################
############################## CSV ##############################

# dataframes = [leuko_df,scc_df, dh_leuko_df, dh_scc_df, hh_df]

# # Concatenate all DataFrames vertically
# combined_df = pd.concat(dataframes)

# csv_path = "C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/two_classes_coded.csv"
# combined_df.to_csv(csv_path, index=False)

##################################################################

############################## Excel ##############################

# writer = pd.ExcelWriter("C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Classifier.xlsx")
# leuko_df.to_excel(writer, sheet_name="Leuko_Spectra")
# scc_df.to_excel(writer, sheet_name="SCC_Spectra")
# dh_leuko_df.to_excel(writer, sheet_name="DH_Leuko_Spectra")
# dh_scc_df.to_excel(writer, sheet_name="DH_SCC_Spectra")
# hh_df.to_excel(writer, sheet_name="Healthy_Spectra")
# writer.save()

##################################################################

In [ ]:
xls = pd.ExcelFile("C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/Classifier.xlsx")
leuko_df = pd.read_excel(xls, "Leuko_Spectra")
scc_df = pd.read_excel(xls, "SCC_Spectra")
dh_leuko_df = pd.read_excel(xls, "DH_Leuko_Spectra")
dh_scc_df = pd.read_excel(xls, "DH_SCC_Spectra")
hh_df = pd.read_excel(xls, "Healthy_Spectra")

# print(leuko_df)

In [ ]:
import math
from scipy.stats import t

def ttest(dataframe1, dataframe2):
    # dataframe.rename(column = {})
    mean_spectra1 = np.mean(dataframe1, axis=0)
    mean_spectra2 = np.mean(dataframe2, axis=0) #.mean(axis=0)

    std1 = np.std(dataframe1, axis=0) #dataframe1[['{}'.format(i) for i in range(0,16)]].std(axis=0)
    std2 = np.std(dataframe2, axis=0) #dataframe2[['{}'.format(i) for i in range(0,16)]].std(axis=0)

    n1 = len(dataframe1)
    n2 = len(dataframe2)

    degrees_of_freedom = n1 + n2 - 2
    t_value = (mean_spectra1 - mean_spectra2) / np.sqrt((np.divide(std1**2, n1) + np.divide(std2**2, n2)))
    p_value = 2 * (1 - t.cdf(abs(t_value), degrees_of_freedom))

    print(f" t_value: {t_value}")
    print(f" p_value: {p_value}")

    alpha = 0.05
    if np.any(p_value < alpha):
        print("The difference between the two sets of spectral data is statistically significant.")
    else:
        print("Not significant")
    return t_value, p_value


# Converting an array to a dataframe
test = ttest(leuko_spectral_img_mean, hh_spectral_img_mean)
test = ttest(scc_spectral_img_mean, hh_spectral_img_mean)



In [ ]:
%matplotlib inline
def mean_spectra(dataframe):
    # dataframe.rename(column = {})
    mean_spectra = dataframe[['{}'.format(i) for i in range(0,16)]].mean(axis=0)
    return(mean_spectra)

#Converting an array to a dataframe
leuko_mean_spectra = mean_spectra(leuko_df)
scc_mean_spectra = mean_spectra(scc_df)
dh_leuko_mean_spectra = mean_spectra(dh_leuko_df)
dh_scc_mean_spectra = mean_spectra(dh_scc_df)
hh_mean_spectra = mean_spectra(hh_df)

# print(leuko_mean_spectra)
# print(scc_mean_spectra)
# print(dh_leuko_mean_spectra)
# print(dh_scc_mean_spectra)

#Plotting the results of the mean spectra of each class
plt.plot(leuko_mean_spectra, linestyle = "solid", color = "royalblue",  label = "Leuko")
plt.plot(scc_mean_spectra, linestyle = "dotted", color = "darkorange", label = "SCC")
plt.plot(dh_leuko_mean_spectra, linestyle = (0, (5, 1)), color = "darkorchid",  label = "Aparently Healthy Leuko")
plt.plot(dh_scc_mean_spectra, linestyle = (0, (3, 5, 1, 5)), color = "maroon",  label = "Aparently Healthy OSCC")
plt.plot(hh_mean_spectra, linestyle = (5, (10, 3)), color= "green", label = "Healthy")
# plt.savefig(f"C:/Users/htic/Desktop/StO2_images/Median.png")
plt.xlim(0,17)
plt.legend()
plt.show()


In [ ]:
df = np.vstack((leuko_mean_spectra, scc_mean_spectra, dh_leuko_mean_spectra, dh_scc_mean_spectra, hh_mean_spectra))
df = df.T
df = pd.DataFrame(df)
csv_path = "C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/matlab.csv"
df.to_csv(csv_path, index=False)

In [ ]:
a = (np.array(leuko_mean_spectra))
print(a.ndim)

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import f_oneway
# Assuming you have the spectral data for each class stored in separate arrays:
class1_spectra = np.array(leuko_mean_spectra).T # Spectra for class 1 (shape: (20, num_bands))
class2_spectra = np.array(scc_mean_spectra).T  # Spectra for class 2 (shape: (20, num_bands))
class3_spectra = np.array(hh_mean_spectra).T # Spectra for class 3 (shape: (20, num_bands))
# Reshape the spectra arrays to be 1-D
# Calculate pairwise spectral angles within each class
class1_angles = []
for i in range(len(class1_spectra)):
    for j in range(i+1, len(class1_spectra)):
        angle = cosine(class1_spectra[i], class1_spectra[j])
        class1_angles.append(angle)

class2_angles = []
for i in range(len(class2_spectra)):
    for j in range(i+1, len(class2_spectra)):
        angle = cosine(class2_spectra[i], class2_spectra[j])
        class2_angles.append(angle)

class3_angles = []
for i in range(len(class3_spectra)):
    for j in range(i+1, len(class3_spectra)):
        angle = cosine(class3_spectra[i], class3_spectra[j])
        class3_angles.append(angle)

mean_angle_class1 = np.mean(class1_angles)
mean_angle_class2 = np.mean(class2_angles)
mean_angle_class3 = np.mean(class3_angles)
# Perform ANOVA on the mean spectral angles
f_statistic, p_value = f_oneway(class1_angles, class2_angles, class3_angles)
print("Mean Spectral Angles:")
print("Class 1:", mean_angle_class1)
print("Class 2:", mean_angle_class2)
print("Class 3:", mean_angle_class3)
print("ANOVA p-value:", p_value)

In [ ]:
from scipy.signal import find_peaks
%matplotlib inline
def perform_peak_analysis(spectra_data):
    # Smooth the spectra data if necessary
    # smoothed_data = smooth_function(spectra_data)  # Replace 'smooth_function' with the appropriate smoothing method

    # Find peaks in the spectra data
    peaks, _ = find_peaks(spectra_data, height=0.51, distance=1) # Adjust 'threshold_value' as needed

    # Retrieve peak positions and intensities
    peak_positions = np.array(range(len(spectra_data)))[peaks]
    peak_intensities = spectra_data[peaks]

    # Perform further analysis on the identified peaks
    # ...

    return peak_positions, peak_intensities

# Perform peak analysis on each dataset
leuko_peak_positions, leuko_peak_intensities = perform_peak_analysis(leuko_mean_spectra)
scc_peak_positions, scc_peak_intensities = perform_peak_analysis(scc_mean_spectra)
dh_leuko_peak_positions, dh_leuko_peak_intensities = perform_peak_analysis(dh_leuko_mean_spectra)
dh_scc_peak_positions, dh_scc_peak_intensities = perform_peak_analysis(dh_scc_mean_spectra)
hh_peak_positions, hh_peak_intensities = perform_peak_analysis(hh_mean_spectra)

# Plot the spectra with identified peaks
plt.plot(leuko_mean_spectra, label='Leuko')
plt.plot(scc_mean_spectra, label='SCC')
plt.plot(dh_leuko_mean_spectra, label='DH Leuko')
plt.plot(dh_scc_mean_spectra, label='DH SCC')
plt.plot(hh_mean_spectra, label='HH')
plt.scatter(leuko_peak_positions, leuko_peak_intensities, color='red', label='Peaks (Leuko)')
plt.scatter(scc_peak_positions, scc_peak_intensities, color='blue', label='Peaks (SCC)')
plt.scatter(dh_leuko_peak_positions, dh_leuko_peak_intensities, color='green', label='Peaks (DH Leuko)')
plt.scatter(dh_scc_peak_positions, dh_scc_peak_intensities, color='orange', label='Peaks (DH SCC)')
plt.scatter(hh_peak_positions, hh_peak_intensities, color='purple', label='Peaks (HH)')
plt.legend()
plt.xlabel('Wavelength/Band')
plt.ylabel('Intensity')
plt.title('Spectra with Identified Peaks')
plt.show()

from scipy.signal import argrelextrema
local_minima_indices = argrelextrema(np.array(leuko_mean_spectra), np.less)[0]
local_minima_values = leuko_mean_spectra[local_minima_indices]

print("Local Minima Indices:", local_minima_indices)
print("Local Minima Values:", local_minima_values)


In [ ]:
# # Assuming your data is stored in a pandas DataFrame named 'df'
# he_df = pd.read_excel(xls, "Healthy Spectra")
# he_data = he_df[he_df['Patho'] == 'Healthy']  # Filter rows with 'Patho' column value equal to 'Leuko'
# band_columns = ['Band {}'.format(i) for i in range(1, 16)]  # Create a list of column names from 'Band 1' to 'Band 16'
# he_band_data = he_data[band_columns]  # Select the columns containing leuko band data
# he_band_data = he_band_data.T
# # Plotting the data
# plt.figure(figsize=(10, 6))  # Set the figure size

# # Iterate over the columns and plot each one
# for column in he_band_data.columns:
#     plt.plot(he_band_data[column], label=column)

# plt.xlabel('Data Points')
# plt.ylabel('Healthy Data')
# plt.title('Healthy Data from Band 1 to Band 16')
# plt.legend()
# plt.grid(True)

# plt.show()

In [ ]:
#Create a dataframe from the list of mean StO2 values
leuko_sto2_df =  pd.DataFrame([np.array(leuko_mean_StO2)])
scc_sto2_df = pd.DataFrame([np.array(scc_mean_StO2)])
dh_leuko_sto2_df = pd.DataFrame([np.array(dh_leuko_mean_StO2)])
dh_scc_sto2_df = pd.DataFrame([np.array(dh_scc_mean_StO2)])
hh_sto2_df = pd.DataFrame([np.array(hh_mean_StO2)])

# To save the datafram to a excel file
writer = pd.ExcelWriter("C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/MSI_StO2.xlsx")
leuko_sto2_df.to_excel(writer, sheet_name="Leuko_StO2")
scc_sto2_df.to_excel(writer, sheet_name="SCC_StO2")
dh_leuko_sto2_df.to_excel(writer, sheet_name="DH_Leuko_StO2")
dh_scc_sto2_df.to_excel(writer, sheet_name="DH_SCC_StO2")
hh_sto2_df.to_excel(writer, sheet_name="Healthy_StO2")
writer.save()

In [ ]:
#R545/R575
def ratio(img, mask):
    band545 =  img[:,:,10] #Corresponds to band 542nm
    band575 = img[:,:,14] #Corresponds to band 575nm

    rat = (band545/band575)
    mask = mask
    masking = cv2.bitwise_and(rat, rat, mask = mask)
    unmasked_area = masking[np.nonzero(masking)]
    # print(f"Mean 545: {np.mean(band545)}, Mean 575: {np.mean(band575)}, Ratio:{np.mean(unmasked_area)}")
    return (np.mean(unmasked_area))

In [ ]:
# Calculate ratios for Leuko images
leukorats = np.array([ratio(img, mask) for img, mask in zip(data_processor.leuko_imgs, data_processor.leuko_masks)])

# Calculate ratios for OSCC images
sccrats = np.array([ratio(img, mask) for img, mask in zip(data_processor.scc_imgs, data_processor.scc_masks)])

# Calculate ratios for DH-Leuko images
dhleukorats = np.array([ratio(img, mask) for img, mask in zip(data_processor.dh_leuko_imgs, data_processor.dh_leuko_masks)])

# Calculate ratios for DH-OSCC images
dhsccrats = np.array([ratio(img, mask) for img, mask in zip(data_processor.dh_scc_imgs, data_processor.dh_scc_masks)])

# Calculate ratios for Healthy images
hhrats = np.array([ratio(img, mask) for img, mask in zip(data_processor.hh_imgs, data_processor.hh_masks)])


In [ ]:
#Creating a datafram of the StO2 vsalues
leuko_ratio_df =  pd.DataFrame([np.array(leukorats)])
scc_ratio_df = pd.DataFrame([np.array(sccrats)])
dh_leuko_ratio_df = pd.DataFrame([np.array(dhleukorats)])
dh_scc_ratio_df = pd.DataFrame([np.array(dhsccrats)])
hh_ratio_df = pd.DataFrame([np.array(hhrats)])

#Saving the dataframe to an excel file
writer = pd.ExcelWriter("C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/MSI_StO2.xlsx")
leuko_ratio_df.to_excel(writer, sheet_name="Leuko_Ratio")
scc_ratio_df.to_excel(writer, sheet_name="SCC_Ratio")
dh_leuko_ratio_df.to_excel(writer, sheet_name="DH_Leuko_Ratio")
dh_scc_ratio_df.to_excel(writer, sheet_name="DH_SCC_Ratio")
hh_ratio_df.to_excel(writer, sheet_name="Healthy_Ratio")
writer.save()


In [ ]:
################ Eculidean Distance between each classes ################
class3_data = leuko_spectral_img_mean  # Class 1 samples (shape: 20x16)
class2_data = scc_spectral_img_mean  # Class 2 samples (shape: 20x16)
class1_data = hh_spectral_img_mean  # Class 3 samples (shape: 20x16)

# Calculate spectral distances between pairs of samples using Euclidean distance
distances_class1 = []
distances_class2 = []
for i in range(len(class1_data)):
    for j in range(len(class2_data)):
        distances_class1.append(euclidean(class1_data[i], class2_data[j]))
    for j in range(len(class3_data)):
        distances_class2.append(euclidean(class1_data[i], class3_data[j]))
# Perform one-way ANOVA
f_value, p_value = f_oneway(distances_class1, distances_class2)
print(f"Mean Distanmce between HH vs OSCC: {np.mean(distances_class1)}, STD: {np.std(distances_class1)}")
print(f"Mean Distanmce between HH vs Leuko: {np.mean(distances_class2)}, STD: {np.std(distances_class2)}")
# Print the results
print("One-way ANOVA results:")
print("F-value:", f_value)
print("p-value:", p_value)

In [ ]:
three_class_df = pd.DataFrame(leuko_spectral_img_mean)


In [ ]:
#Eculidean Distance between each classes
class3_data = leuko_spectral_img_mean  # Class 1 samples (shape: 20x16)
class2_data = scc_spectral_img_mean  # Class 2 samples (shape: 20x16)
class1_data = hh_spectral_img_mean  # Class 3 samples (shape: 20x16)

# Calculate spectral distances between pairs of samples using Euclidean distance
fvals_class12 = []
fvals_class23 = []

pvals_class12 = []
pvals_class23 = []
for i in range(len(class1_data)):
    for j in range(len(class2_data)):
        f_value, p_value = f_oneway(class1_data, class2_data)
        fvals_class12.append(f_value)
        pvals_class12.append(p_value)
    for j in range(len(class3_data)):
        f_value, p_value = f_oneway(class3_data, class2_data)
        fvals_class23.append(f_value)
        pvals_class23.append(p_value)
# Perform one-way ANOVA
# f_value, p_value = f_oneway(distances_class1, distances_class2)
print(f"ANOVA between H vs OSCC:\n p_vals =\n {pvals_class12[5]},\n f_vals =\n {fvals_class12[5]}")
print(f"ANOVA between H vs L:\n p_vals =\n {pvals_class23[5]},\n f_vals = \n{fvals_class23[5]}")



# for i in range(len(fvals_class12)):
#     print(np.mean(pvals_class12[i]))
#     print(np.mean(pvals_class23[i]))

In [ ]:
from statsmodels.multivariate.manova import MANOVA

In [ ]:
combined_df['Patho'].values

In [ ]:

# Assuming your DataFrame is named 'df' with columns: 'Band1', 'Band2', ..., 'Band16' and 'Class'

# Extract the dependent variable matrix
y_matrix = combined_df.loc[:, '0':'15'].values

# Extract the independent variable matrix
cl = combined_df['Patho'].values

# Perform MANOVA
manova = MANOVA(y_matrix, groups=cl)

# Fit the MANOVA model
result = manova.fit()

In [ ]:
#Eculidean Distance between each classes
def euclidean_distance(class1_data, class2_data):
    distance_inclasses = []
    for i in range(len(class1_data)):
        for j in range(len(class2_data)):
            distance_inclasses.append(euclidean(class1_data[i], class2_data[j]))

    return distance_inclasses

ed_hh_vs_scc = euclidean_distance(hh_spectral_img_mean, scc_spectral_img_mean)
print(f"Mean Distanmce between HH vs OSCC: {np.mean(ed_hh_vs_scc)}, STD: {np.std(ed_hh_vs_scc)}")

ed_leuko_vs_hh = euclidean_distance(leuko_spectral_img_mean, hh_spectral_img_mean)
print(f"Mean Distanmce between HH vs Leuko: {np.mean(ed_leuko_vs_hh)}, STD: {np.std(ed_leuko_vs_hh)}")

ed_leuko_vs_dhleuko = euclidean_distance(leuko_spectral_img_mean, dh_leuko_spectral_img_mean)
print(f"Mean Distanmce between DH-Leuko vs Leuko: {np.mean(ed_leuko_vs_dhleuko)}, STD: {np.std(ed_leuko_vs_dhleuko)}")

ed_scc_vs_dhscc = euclidean_distance(scc_spectral_img_mean, dh_scc_spectral_img_mean)
print(f"Mean Distanmce between DH-SCC vs OSCC: {np.mean(ed_scc_vs_dhscc)}, STD: {np.std(ed_scc_vs_dhscc)}")

ed_oscc_vs_dhleuko = euclidean_distance(scc_spectral_img_mean, dh_leuko_spectral_img_mean)
print(f"Mean Distanmce between DH-Leuko vs OSCC: {np.mean(ed_oscc_vs_dhleuko)}, STD: {np.std(ed_oscc_vs_dhleuko)}")

ed_oscc_vs_leuko = euclidean_distance(scc_spectral_img_mean, leuko_spectral_img_mean)
print(f"Mean Distance between Leuko vs OSCC: {np.mean(ed_oscc_vs_leuko)}, STD: {np.std(ed_oscc_vs_leuko)}")


In [ ]:
#ANOVA between bands of different class - StO2
class3_data = np.array(leuko_spectral_img_mean[:21])  # Class 1 samples (shape: 20x16)
class2_data = np.array(scc_spectral_img_mean)  # Class 2 samples (shape: 20x16)
class1_data = np.array(hh_spectral_img_mean[:21])  # Class 3 samples (shape: 20x16)

# Reshape the data into a 2D array (bands as columns, observations as rows)
data = np.concatenate((class3_data, class2_data, class1_data), axis=1).T
# Perform ANOVA
f_value, p_value = f_oneway(class2_data, class3_data, class1_data)
print("F-value:", f_value)
print("p-value:", p_value)

In [ ]:
#Scatter plot of R545/R575 with all classes -StO2
y1 = np.arange(1, len(leukorats)+1)
y2 = np.arange(1, len(sccrats)+1)
y3 = np.arange(1, len(dhleukorats)+1)
y4 = np.arange(1, len(dhsccrats)+1)
y5 = np.arange(1, len(hhrats)+1)

plt.scatter(y1, leukorats, color = "royalblue", label = "Leuko")
plt.scatter(y2, sccrats, color = "darkorange", label = "SCC")
plt.scatter(y3, dhleukorats, color = "darkorchid", label = "DH-Leuko")
plt.scatter(y4, dhsccrats, color = "maroon", label = "DH-Leuko")
plt.scatter(y5, hhrats, color = "green", label = "Healthy")

plt.title("Scatter Plot of R545/R575")
plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1))

In [ ]:
#Scatter plot of R545/R575 with all classes
y1 = np.arange(1, len(leukorats)+1)
y2 = np.arange(1, len(sccrats)+1)
y3 = np.arange(1, len(dhleukorats)+1)
y4 = np.arange(1, len(dhsccrats)+1)
y5 = np.arange(1, len(hhrats)+1)

plt.scatter(y1, leukorats, color = "royalblue", label = "Leuko")
plt.scatter(y5, hhrats, color = "seagreen", label = "Healthy")
plt.axhline(np.mean(leukorats), color = "blue", linestyle = "--", label="Leuko")
plt.axhline(np.mean(hhrats), color = "seagreen", linestyle = "--", label="Healthy")

plt.title("R545/R575 HH vs Leuko")
plt.legend()

In [ ]:
y1 = np.arange(1, len(leukorats)+1)
y2 = np.arange(1, len(sccrats)+1)
y3 = np.arange(1, len(dhleukorats)+1)
y4 = np.arange(1, len(dhsccrats)+1)
y5 = np.arange(1, len(hhrats)+1)

plt.scatter(y2, sccrats, color = "darkorange",  label = "SCC")
plt.scatter(y5, hhrats, color = "seagreen", label = "Healthy")

plt.axhline(np.mean(sccrats), color = "darkorange", linestyle = "--", label="SCC")
plt.axhline(np.mean(hhrats), color = "seagreen", linestyle = "--", label="Healthy")

plt.title("R545/R575 SCC vs Healthy")
plt.legend()

In [ ]:
scc_std = np.std(sccrats, axis=0)
lower_cutoff_scc = np.mean(sccrats) - scc_std
upper_cutoff_scc = np.mean(sccrats) + scc_std

print(np.mean(sccrats))
print(lower_cutoff_scc)

hh_std = np.std(hhrats, axis=0)
lower_cutoff_hh = np.mean(hhrats) - hh_std
upper_cutoff_hh = np.mean(hhrats) + hh_std

print(np.mean(hhrats))
print(upper_cutoff_hh)

# Combine the data and corresponding labels
scc_gt = np.where(sccrats > lower_cutoff_scc, 1, 0)
scc_gt = np.array([1 if x > (lower_cutoff_scc)  else 0 for x in (sccrats)])
hh_gt = np.array([1 if (x > (lower_cutoff_hh) and x < (upper_cutoff_hh)) else 0 for x in (hhrats)])
# print(scc_gt, hh_gt)
y_true = np.concatenate((scc_gt, hh_gt))
y_scores = np.concatenate([sccrats, hhrats])

print("y_true", y_true)
print("y_score", y_scores)

# Calculate the false positive rate, true positive rate, and thresholds
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
print(f"Fpr = {fpr}, tpr = {tpr}, thresholds = {thresholds}")

# Calculate the AUC (Area Under the Curve)
auc_score = auc(fpr, tpr)
print(f"{auc_score}")

# Plot the ROC curve
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], color='lightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('Receiver Operating Characteristic (ROC) Healthy vs OSCC')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Cnnfusing matrix 

# leuko_std = np.std(leukorats, axis=0)
# lower_cutoff_leuko = np.mean(leukorats) - leuko_std
# upper_cutoff_leuko = np.mean(leukorats) + leuko_std

# print(np.mean(leukorats))
# print(leuko_std)

# hh_std = np.std(hhrats, axis=0)
# lower_cutoff_hh = np.mean(hhrats) - hh_std
# upper_cutoff_hh = np.mean(hhrats) + hh_std

# print(np.mean(hhrats))
# print(hh_std)

# # Combine the data and corresponding labels
# # leuko_gt = np.where(leuko_gt > lower_cutoff_scc, 1, 0)
# leuko_gt = np.array([1 if x > (lower_cutoff_leuko)  else 0 for x in (leukorats)])
# hh_gt = np.array([1 if (x > (lower_cutoff_hh) and x < (upper_cutoff_hh)) else 0 for x in (hhrats)])
# # print(scc_gt, hh_gt)
# y_true = np.concatenate((leuko_gt, hh_gt))
# y_scores = np.concatenate([leukorats, hhrats])

# print("y_true", y_true)
# print("y_score", y_scores)

# y_scores_positive = y_scores[y_true == 1]

# # Determine the threshold for classifying as positive based on the desired false positive rate (fpr)
# desired_fpr = 0.2  # Set the desired false positive rate here
# threshold_index = np.argmax(fpr >= desired_fpr)  # Find the index where fpr first exceeds the desired rate
# threshold = thresholds[threshold_index]

# # Calculate the predicted labels based on the threshold
# y_pred = np.array([1 if score > threshold else 0 for score in y_scores_positive])

# # Create the confusion matrix
# cm = confusion_matrix(y_true, y_pred)

# # Plot the confusion matrix as a heatmap
# plt.imshow(cm, cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.colorbar()
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.xticks([0, 1], ['Negative', 'Positive'])
# plt.yticks([0, 1], ['Negative', 'Positive'])
# plt.show()

In [ ]:
#Wrong measurements
# leuko_std = np.std(leukorats)
# lower_cutoff_leuko = np.mean(leukorats) - leuko_std
# upper_cutoff_leuko = np.mean(leukorats) + leuko_std

# print(np.mean(leukorats))
# print(leuko_std)

# hh_std = np.std(hhrats)
# lower_cutoff_hh = np.mean(hhrats) - hh_std
# upper_cutoff_hh = np.mean(hhrats) + hh_std

# print(np.mean(hhrats))
# print(hh_std)

# # Combine the data and corresponding labels
# # leuko_gt = np.where(leuko_gt > lower_cutoff_scc, 1, 0)
# leuko_gt = np.array([1 if x > (lower_cutoff_leuko)  else 0 for x in (leukorats)])
# hh_gt = np.array([1 if (x > (lower_cutoff_hh) and x < (upper_cutoff_hh)) else 0 for x in (hhrats)])
# # print(scc_gt, hh_gt)
# y_true = np.concatenate((leuko_gt, hh_gt))
# y_scores = np.concatenate([leukorats, hhrats])

# # Plot the ROC curve
# plt.figure(figsize=(6, 6))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % auc_score)
# plt.plot([0, 1], [0, 1], color='lightblue', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate (1 - Specificity)')
# plt.ylabel('True Positive Rate (Sensitivity)')
# plt.title('Receiver Operating Characteristic (ROC) Healthy vs Leuko')
# plt.legend(loc="lower right")
# plt.show()


In [ ]:
# #LDA for StO2 - Not used

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.decomposition import PCA

# # Generate some random data for demonstration
# np.random.seed(0)
# class1 = np.array([leukorats]) #np.random.multivariate_normal(mean=[1, 1], cov=[[1, 0.5], [0.5, 1]], size=50)
# class2 = np.array([hhrats]) #np.random.multivariate_normal(mean=[-1, -1], cov=[[1, 0.5], [0.5, 1]], size=50)
# class3 = np.array([sccrats]).squeeze(0)

# class1 = class1[0][:21]
# class2 = class2[0][:21]
# print(class1.shape, class2.shape, class3.shape)
# class1 = class1.reshape(-1,1)
# class2 = class2.reshape(-1,1)
# class3 = class3.reshape(-1,1)

# # Combine the reshaped arrays into a single array
# data = np.concatenate((class1, class2, class3))

# # Create the target labels for each class
# target = np.concatenate((np.zeros_like(class1), np.ones_like(class2), 2 * np.ones_like(class3)))

# # Perform LDA
# lda = LinearDiscriminantAnalysis(n_components=1)
# X_lda = lda.fit_transform(data, target)

# # Plot the LDA projection
# plt.scatter(X_lda[:len(class1)], np.zeros_like(class1), color='red', label='Class 1')
# plt.scatter(X_lda[len(class1):len(class1) + len(class2)], np.zeros_like(class2), color='blue', label='Class 2')
# plt.scatter(X_lda[len(class1) + len(class2):], np.zeros_like(class3), color='green', label='Class 3')

# plt.xlabel('LDA Projection')
# plt.title('LDA Projection for Three Classes')
# plt.legend()
# plt.show()

In [ ]:
#### SAM neatly done is another file ###
# def SAM(images, rgbs, masked_data, mean_spec, patho):
#     if patho == 0:
#         rgbs_name_list = data_processor.leuko_rgbs_name
#         masks_name_list = data_processor.leuko_masks_name
#         data_name_list = data_processor.leuko_data_name
#     elif patho == 1:
#         rgbs_name_list = data_processor.scc_rgbs_name
#         masks_name_list = data_processor.scc_masks_name
#         data_name_list = data_processor.scc_data_name
#     elif patho == 2:
#         rgbs_name_list = data_processor.dh_leuko_rgbs_name
#         masks_name_list = data_processor.dh_leuko_masks_name
#         data_name_list = data_processor.dh_leuko_data_name
#     elif patho == 3:
#         rgbs_name_list = data_processor.dh_scc_rgbs_name
#         masks_name_list = data_processor.dh_scc_masks_name
#         data_name_list = data_processor.dh_scc_data_name
#     elif patho == 4:
#         rgbs_name_list = data_processor.hh_rgbs_name
#         masks_name_list = data_processor.hh_masks_name
#         data_name_list = data_processor.hh_data_name

#     ref_vector = np.array(mean_spec)
#     print(ref_vector)
#     for k in range(len(images)):
#         img = images[k]
#         sam_img = np.zeros((img.shape[0], img.shape[1]))
        
#         # Compute the dot product between each pixel's spectrum and the reference vector
#         dot_product = np.sum(img * ref_vector, axis=-1)
        
#         # Compute the norms of each pixel's spectrum and the reference vector
#         norm_product = np.linalg.norm(img, axis=-1) * np.linalg.norm(ref_vector)
        
#         # Compute the angle between each pixel's spectrum and the reference vector
#         angle = np.arccos(dot_product / norm_product)
#         degree = np.degrees(angle)
#         # print(angle.shape, degree.shape)
        
#         # Update the sam_img based on the angle threshold
#         sam_img[degree < 20] = 255
#         plt.figure(figsize=(15, 30))
#         plt.subplot(1, 3, 1), plt.imshow(rgbs[k]), plt.title(f"RGB {rgbs_name_list[k]}")
#         plt.subplot(1, 3, 3), plt.imshow(sam_img, cmap='gray'), plt.title(f"SAM classification of {data_name_list[k]}")
#         plt.subplot(1, 3, 2), plt.imshow(masked_data[k], cmap='gray'), plt.title(f"masked_data {masks_name_list[k]}")
#         plt.show()

# leuko_sam = SAM(data_processor.leuko_imgs, data_processor.leuko_rgbs, data_processor.leuko_masks, leuko_mean_spectra, 0)
# scc_sam = SAM(data_processor.scc_imgs, data_processor.scc_rgbs, data_processor.scc_masks, scc_mean_spectra, 1)
# scc_sam = SAM(data_processor.scc_imgs, data_processor.scc_rgbs, data_processor.scc_masks, scc_mean_spectra, 1)

# import numpy as np
# from scipy.spatial.distance import cdist
# def select_reference_spectra(ref_spectra, n):
#     """
#     Function to select reference spectra based on the specified criteria.
#     Parameters:
#     - ref_spectra (ndarray or list): List of reference spectra.
#     - n (int): Number of reference spectra to select.
#     Returns:
#     - selected_refs (list): List of selected reference spectra.
#     """
#     if n == 1:
#         return ref_spectra
#     else:
#         if isinstance(ref_spectra, list):
#             ref_spectra = np.array(ref_spectra)
#         selected_refs = []
#         for i in range(n):
#             if i == 0:
#                 # Select the median spectrum as the first reference spectrum
#                 selected_refs.append(ref_spectra)
#             else:
#                 # Calculate distances between the current selected reference spectra and remaining reference spectra
#                 distances = cdist(selected_refs, ref_spectra)
#                 # Find the index of the closest spectrum to the median spectrum
#                 closest_idx = np.argmin(np.min(distances, axis=0))
#                 print(closest_idx)
#                 # Add the closest spectrum to selected_refs
#                 selected_refs.append(ref_spectra[closest_idx])
#         return selected_refs
# import numpy as np
# from scipy.spatial.distance import cosine

# def spectral_angle_mapper(ref_vectors, taregt_image):
#     for i in range(ref_vectors.shape[0]):
        
#         ref_vector = np.array(ref_vectors[i])
#         print(ref_vector)
#         for k in range(len(taregt_image)):
#             img = taregt_image
#             img_reshaped = np.reshape(img, (-1, img.shape[2]))
#             sam_img = np.zeros((img.shape[0], img.shape[1]))
            
#             # Compute the dot product between each pixel's spectrum and the reference vector
#             dot_product = np.sum(img_reshaped * ref_vector, axis=-1)
#             dot_product = dot_product.reshape((taregt_image.shape[0], taregt_image.shape[1]))
#             # Compute the norms of each pixel's spectrum and the reference vector
#             norm_product = np.linalg.norm(img, axis=-1) * np.linalg.norm(ref_vector)
            
#             # Compute the angle between each pixel's spectrum and the reference vector
#             angle = np.arccos(dot_product / norm_product)
#             degree = np.degrees(angle)
#             # print(angle.shape, degree.shape)
            
#             # Update the sam_img based on the angle threshold
#             sam_img[degree < 25] = 255

#     return np.array(sam_img)
# # selected_refs_exhaustive = select_reference_spectra(ref_spectra, mode='exhaustive')
# # print("Selected reference spectra (exhaustive mode):")
# # print(selected_refs_exhaustive)
# # Select reference spectra using the 'percent' mode (80% of reference spectra)
# imgs =  data_processor.leuko_imgs
# ref_vectors = leuko_spectral_img_mean
# # print((leuko_spectral_img_mean))
# # print(ref_vectors.shape)

# selected_refs = select_reference_spectra(ref_vectors,20)
# print("Selected reference spectra:")
# print(selected_refs)

# sam_img = spectral_angle_mapper(selected_refs,imgs[6])

# print("Selected reference spectra (percent mode):")
# print(selected_refs)



In [ ]:
def calculate_sam_angle(ref_spectrum1, ref_spectrum2):
    # Calculate the dot product of the normalized spectra
    dot_product = np.dot(ref_spectrum1, ref_spectrum2)
    norm_product = np.dot(np.linalg.norm(ref_spectrum1), np.linalg.norm(ref_spectrum2))

    # Calculate the SAM angle in radians
    sam_angle = np.arccos(dot_product / norm_product)
    degree = np.degrees(sam_angle)
    return degree
    
print("Angles Between Mean Spectras")
#SAM b/w OSS vs Healthy
ssc_vs_healthy = calculate_sam_angle(np.array(scc_mean_spectra), np.array(hh_mean_spectra))
print(f"OSCC vs Healthy - {ssc_vs_healthy:.2f}")

#SAM b/w OSS vs Leuko
ssc_vs_leuko = calculate_sam_angle(np.array(scc_mean_spectra),np.array(leuko_mean_spectra))
print(f"OSCC vs Leuko - {ssc_vs_leuko:.2f}")

#SAM b/w Leuko and DH-Leuko
leuko_vs_dhleuko = calculate_sam_angle(np.array(dh_leuko_mean_spectra),np.array(leuko_mean_spectra))
print(f"Leuko vs DH-Leuko - {leuko_vs_dhleuko:.2f}")

#SAM b/w OSCC vs DH-OSCC
scc_vs_dhscc = calculate_sam_angle(np.array(scc_mean_spectra), np.array(dh_scc_mean_spectra))
print(f"OSCC vs DH-SCC - {scc_vs_dhscc:.2f}")

#SAM b/w Healthy vs Leuko
hh_vs_leuko = calculate_sam_angle(np.array(hh_mean_spectra), np.array(leuko_mean_spectra))
print(f"Healthy vs Leuko - {hh_vs_leuko:.2f}")

#SAM b/w Healthy and DH-Leuko
hh_vs_dhleuko = calculate_sam_angle(np.array(hh_mean_spectra), np.array(dh_leuko_mean_spectra))
print(f"Healthy vs DH-Leuko - {hh_vs_dhleuko:.2f}")

#SAM b/w Healthy and DH-SCC
hh_vs_dhscc = calculate_sam_angle(np.array(hh_mean_spectra), np.array(dh_scc_mean_spectra))
print(f"Healthy vs DH-SCC - {hh_vs_dhscc:.2f}")

#SAM b/w SCC vs DH-Leuko
scc_vs_dhleuko = calculate_sam_angle(np.array(scc_mean_spectra), np.array(dh_leuko_mean_spectra))
print(f"OSCC vs DH-Leuko - {scc_vs_dhleuko:.2f}")

In [ ]:
def std_spectra(dataframe):
    std_spectra = dataframe[['Band {}'.format(i) for i in range(1, 17)]].std(axis=0)
    return(std_spectra)

leuko_std = np.array(std_spectra(leuko_df))
scc_std = np.array(std_spectra(scc_df))
dhleuko_std = np.array(std_spectra(dh_leuko_df))
dhscc_std = np.array(std_spectra(dh_scc_df))
hh_std = np.array(std_spectra(hh_df))


leuko_pstd = np.array(leuko_mean_spectra) + leuko_std
# print(f"Leuko + Std: {calculate_sam_angle(leuko_pstd, np.array(leuko_mean_spectra)):.2f}")

scc_pstd = np.array(scc_mean_spectra) + scc_std
# print(f"OSCC + Std: {calculate_sam_angle(scc_pstd, np.array(scc_mean_spectra)):.2f}")

dhleuko_pstd = np.array(dh_leuko_mean_spectra) + dhleuko_std
# print(f"DH-Leuko + Std: {calculate_sam_angle(dhleuko_pstd, np.array(dh_leuko_mean_spectra)):.2f}")

dhscc_pstd = np.array(dh_scc_mean_spectra) + dhscc_std
# print(f"DH-SCC + Std: {calculate_sam_angle(dhscc_pstd, np.array(dh_scc_mean_spectra)):.2f}")

hh_pstd = np.array(hh_mean_spectra) + hh_std 
# print(f"HH + Std: {calculate_sam_angle(hh_pstd, np.array(hh_mean_spectra)):.2f}")

print("Angles between standard deviation")
print(f"OSCC vs Healthy: {calculate_sam_angle(scc_pstd, hh_pstd):.2f}")
print(f"OSCC vs Leuko: {calculate_sam_angle(scc_pstd, leuko_pstd):.2f}")
print(f"Leuko vs DH-Leuko: {calculate_sam_angle(leuko_pstd, dhleuko_pstd):.2f}")
print(f"OSCC vs DH-OSCC: {calculate_sam_angle(scc_pstd, dhscc_pstd):.2f}")
print(f"Healthy vs Leuko: {calculate_sam_angle(hh_pstd, leuko_pstd):.2f}")
print(f"Healthy vs DH-Leuko: {calculate_sam_angle(hh_pstd, dhleuko_pstd):.2f}")
print(f"Healthy vs DH-OSCC: {calculate_sam_angle(hh_pstd, dhscc_pstd):.2f}")
print(f"OSCC vs DH-Leuko: {calculate_sam_angle(scc_pstd, dhleuko_pstd):.2f}")


In [ ]:
# #SVM to draw lines. Code not used

# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.svm import SVC

# # Assuming you have three feature arrays hh, leu, scc
# hh = np.array([84.03368, 84.79525, 86.316666, 82.443436, 83.835724, 83.80833, 85.645744, 83.49889, 83.17439, 83.87311, 82.7982, 82.99992, 82.850876, 84.42748, 83.68328, 83.567345, 83.7835, 83.07346, 83.34416, 85.5577, 86.881676])
# leu = np.array([87.46472, 90.79058, 86.25486, 85.30591, 86.49944, 84.43412, 84.359436, 88.74258, 84.02049, 87.82078, 87.72358, 90.95051, 84.205864, 86.55514, 89.19359, 87.24315, 90.86952, 85.63798, 88.197136, 88.232445, 86.7485])
# scc = np.array([94.2164, 80.30851, 95.761826, 95.877716, 93.5911, 92.56847, 93.30519, 89.8591, 93.17879, 99.16925, 91.27754, 88.54665, 83.31736, 83.36424, 95.64234, 91.12258, 88.02359, 86.08707, 88.83949, 93.723694, 97.50155])

# # Combine the feature arrays into a single 2D array
# X = np.concatenate((hh[:, np.newaxis], leu[:, np.newaxis], scc[:, np.newaxis]), axis=1)
# print(X)

# # Create the label array
# y = np.array(np.concatenate((np.ones(len(hh)), 2 * np.ones(len(leu)), 3 * np.ones(len(scc)))))
# print(y)
# # Create a SVM classifier with a linear kernel
# classifier = SVC(kernel='linear')

# # fitting x samples and y classes 
# clf.fit(x, y) 


# # Create a list to store the decision boundary lines
# boundary_lines = []

# # Create combinations of classes for separation
# class_combinations = [(1, 2), (1, 3), (2, 3)]

# # Train binary classifiers for each class combination
# for class_pair in class_combinations:
#     # Create a boolean array that is the same size as X
#     boolean_array = np.where(y == class_pair[0] or y == class_pair[1]).all()

# # Index X with the boolean array
#     X_pair = X[boolean_array]
#     # Select samples and labels for the current class pair
#     X_pair = X[np.logical_or(y == class_pair[0], y == class_pair[1])]
#     y_pair = y[np.logical_or(y == class_pair[0], y == class_pair[1])]
    
#     # Train the binary classifier
#     classifier.fit(X_pair, y_pair)
    
#     # Get the coefficients and intercept
#     coef = classifier.coef_[0]
#     intercept = classifier.intercept_[0]
    
#     # Generate x values for the decision boundary line
#     x_boundary = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
    
#     # Calculate y values for the decision boundary line
#     y_boundary = -(coef[0] * x_boundary + intercept) / coef[1]
    
#     # Store the decision boundary line
#     boundary_lines.append((x_boundary, y_boundary))

# # Plot the data points
# plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis')

# # Plot the decision boundary lines
# for boundary_line in boundary_lines:
#     plt.plot(boundary_line[0], boundary_line[1], color='red', label='Decision Boundary')

# # Set the labels and title
# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.title('Decision Boundaries')

# # Show the legend
# plt.legend()

# # Show the plot
# plt.show()


In [ ]:
def stoo(img):
    attmat = np.array([[52096,49708],[49172,43340]])

    lnR544 = np.divide(img[:,:,10], 49172)  #changing reflectance to absorbance
    lnR572 = np.divide(img[:,:,14], 52096)  #800

    has_inf = np.isinf(lnR544).any()

    if has_inf:
        print("The image contains inf values.")
 
    lnR544 = np.nan_to_num(lnR544)
    lnR572 = np.nan_to_num(lnR572)

    # hbO2 = []
    # Hbb = []
    lnR544 = lnR544.reshape(270*510)
    lnR572 = lnR572.reshape(270*510)
    
    y = np.linalg.lstsq(attmat, [lnR544, lnR572])[0]
    hbo2 = y[0]
    hb = y[1]
    hbO2 = hbo2.reshape(270,510)
    Hbb = hb.reshape(270,510)

    #normalizing Hb, Hb02 values before calculating st02
    n_Hb = ((Hbb- Hbb.min())/(Hbb.max()-Hbb.min()))*100
    n_HbO2 = ((hbO2- hbO2.min())/(hbO2.max()-hbO2.min()))*100

    StO2 = n_HbO2/(n_Hb+n_HbO2)
    n_StO2 = ((StO2 - StO2.min()))/(StO2.max() - StO2.min())      
    return n_Hb, n_HbO2, n_StO2



In [ ]:
mean = []


In [ ]:
# 
i = 8
img = data_processor.hh_imgs[i]
name = data_processor.hh_data_name[i]
rgb_img = data_processor.hh_rgbs[i]
# mask = data_processor.scc_masks[i]
Hb, HbO2, StO2 = stoo(img)

# To pick points
%matplotlib tk
fig = plt.figure(figsize=(20,30))

ptx = []
pty = []

def onclick(event):
    ix, iy = event.ydata, event.xdata
    ptx.append(ix) #rows
    pty.append(iy) #cols

def on_close(event):
    print('Closed Figure!')
    
#r,g,b = img[:,:,15], img[:,:,7], img[:,:,0]
# rgb_img = np.stack((r,g,b),axis=2)

cid = fig.canvas.mpl_connect('button_press_event', onclick)
fig.canvas.mpl_connect('close_event', on_close)
plt.imshow(rgb_img), plt.title('Select the reference region')
plt.show()

In [ ]:
%matplotlib inline
print("Points Selected",ptx,pty)   
ptx = [int(i) for i in ptx]
pty = [int(i) for i in pty]
print(ptx,pty)

x_len = len(ptx)
x_len = len(ptx)
copy = np.copy(rgb_img)

for i in range(0,len(ptx)-1,2):
    x1 = pty[i]
    y1 = ptx[i]
    x2 = pty[i+1]
    y2 = ptx[i+1]
    cv2.rectangle(copy,(x1,y1),(x2,y2),(255,255,255),2)
    # if i==0:
    #     cv2.putText(copy, "Disease ROI", (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)
    # else:
    #     cv2.putText(copy, "Aparently healthy ROI",(x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)


for i in range(0,len(ptx),2):
    ref_roi_rgb = rgb_img[ptx[i]:ptx[i+1],pty[i]:pty[i+1],:]
    ref_roi_msi = img[ptx[i]:ptx[i+1],pty[i]:pty[i+1],:]
    mean.append(np.mean(ref_roi_msi))

# leuko_list = [data_processor.leuko_imgs[21], data_processor.leuko_imgs[20], data_processor.leuko_imgs[46] ]
# scc_list = [data_processor.scc_imgs[0], data_processor.scc_imgs[9], data_processor.scc_imgs[4]]


In [ ]:
# plt.figure(figsize=(6,5))
plt.imshow(copy), plt.axis("off")
plt.savefig(f"C:/Users/htic/Desktop/StO2_images/Results/RGB_{name}.png")
print(f"StO2 in unhealthy region: {mean[24]} StO2: {name}")
      # in healthy region:{mean[23]} {name}")

In [ ]:
copy = StO2
plt.figure(figsize=(6,5))
for i in range(0,len(ptx)-1,2):
    x1 = pty[i]
    y1 = ptx[i]
    x2 = pty[i+1]
    y2 = ptx[i+1]
    cv2.rectangle(copy,(x1,y1),(x2,y2),(255,255,255),2)
    # if i==1:
    #     cv2.putText(copy, "Disease ROI", (x1, y1-5), cv2.FONT_HERSHEY_PLAIN, 1, (36,255,12), 1)
    # else:
    #     cv2.putText(copy, "Apparently Healthy ROI",(x1, y1-5), cv2.FONT_HERSHEY_PLAIN, 1, (36,255,12), 1)
plt.imshow(StO2, cmap = "jet"), plt.axis("off")
plt.clim(0.5,0.725)
plt.colorbar(orientation="horizontal")
plt.tight_layout()
plt.savefig(f"C:/Users/htic/Desktop/StO2_images/Results/StO2_{name}.png")
print(mean)

In [ ]:
plt.figure(figsize=(3.5, 7))

plt.subplot(2, 1, 1)
plt.imshow(copy)
plt.axis("off")

plt.subplot(2, 1, 2)
plt.imshow(StO2, cmap="jet")
plt.axis("off")
plt.clim(0, 0.75)
plt.colorbar(orientation="horizontal")

plt.subplots_adjust(hspace=0.2)  # Optional, to improve spacing between subplots
plt.show()

In [ ]:
# plt.figure(figsize=(3.5, 2))
fig, ax= plt.subplots(1,2, figsize = (3.5,2))
ax[0].imshow(copy), plt.axis('off')
ax[1].imshow(StO2, cmap = "jet"), plt.axis("Off"), plt.colorbar(orientation = "horizontal ")